# Test the BartSpektro architecture
we've done some changes to the BART architecture and need to test it to see if it's working properly.

### Curent changes to test
The model is now having different encoder and decoder sequence lengths. These are modifiable by the encoder_max_position_embeddings (maybe, bcs it doesn't seem to be used anywhere) and decoder_max_position_embeddings (this is at least used to create the position embedding matrix with the correct shape).

Also we deleted the max_position_embeddings from the config. It looks like it's not used anywhere now, but it might pop up somewhere.

TASK: 
- if there is sth that controls the sequence length, FIND IT
- if there is no actual parameter controling the sequence length in both encoder and decoder, make sure it's true
________________________________
### Results

#### Important architectural parameters
- d_model - embedding size
- max_log_id - encoder positional embedding dimension
- max_mz => encoder embedding dimension
- vocab_size => decoder embedding dimension    AND    modeling head dimension
- decoder_max_position_embeddings => decoder positional embedding dimension

not needed:
- encoder_max_position_embeddings
- seq_len
- ...
!! padding is not needed!

In [2]:
import sys
sys.path.append('..')

from data_utils import SpectroDataset, SpectroDataCollator, build_single_datapipe
from bart_spektro.modeling_bart_spektro import BartSpektroForConditionalGeneration
from bart_spektro.configuration_bart_spektro import BartSpektroConfig
from bart_spektro.selfies_tokenizer import hardcode_build_selfies_tokenizer
from tokenizers import Tokenizer
from train_bart import get_spectro_config, build_tokenizer
from torchsummary import summary

import torch

[2024-02-07 21:43:12,800] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
model_args = {
    "separate_encoder_decoder_embeds": True,
    "max_mz": 500,
    "encoder_seq_len": 300,
    "decoder_seq_len": 400,
    "encoder_layers": 12,
    "decoder_layers": 12,
    "encoder_attention_heads": 16,
    "decoder_attention_heads": 16,
    "encoder_ffn_dim": 4096,
    "decoder_ffn_dim": 4096,
}
tokenizer = build_tokenizer("../tokenizer/bbpe_tokenizer/bart_bbpe_tokenizer_1M_mf3.model")
config = get_spectro_config(model_args, tokenizer)

In [4]:
model = BartSpektroForConditionalGeneration(config)

In [5]:
pipe = build_single_datapipe("../deprecated/NIST_split_filip_smiUpTo100/valid.jsonl", True)
loader = torch.utils.data.DataLoader(pipe, batch_size=2, collate_fn=SpectroDataCollator())

/home/xhajek9/gc-ms_bart/notebooks/../data_utils.py:108: UserWarning: SHUFFLE and its buffer_size are IGNORED if either not specified
  warnings.warn("SHUFFLE and its buffer_size are IGNORED if either not specified")


In [6]:
inp = next(iter(loader))

In [7]:
enc_tensor_len = 2
dec_tensor_len = 6
inp["input_ids"] = (torch.cat((inp["input_ids"], inp["input_ids"], inp["input_ids"]), dim=1)[:, :enc_tensor_len]).contiguous()
# inp["attention_mask"] = (torch.cat((inp["attention_mask"], inp["attention_mask"], inp["attention_mask"]), dim=1)[:, :enc_tensor_len]).contiguous()
inp.pop("attention_mask")
inp["position_ids"] = (torch.cat((inp["position_ids"], inp["position_ids"], inp["position_ids"]), dim=1)[:, :enc_tensor_len]).contiguous()
inp["decoder_attention_mask"] = (torch.cat((inp["decoder_attention_mask"], inp["decoder_attention_mask"], inp["decoder_attention_mask"]), dim=1)[:, :dec_tensor_len]).contiguous()
inp["labels"] = (torch.cat((inp["labels"], inp["labels"], inp["labels"]), dim=1)[:, :dec_tensor_len]).contiguous()

for name, t in inp.items():
    print(name, + (24 - len(name))*" ", t.shape)

input_ids                 torch.Size([2, 2])
position_ids              torch.Size([2, 2])
decoder_attention_mask    torch.Size([2, 6])
labels                    torch.Size([2, 6])


In [8]:
out = model.forward(**inp)
print(out.logits.shape)
print(out.encoder_last_hidden_state.shape)
out


torch.Size([2, 6, 1240])
torch.Size([2, 2, 1024])


Seq2SeqLMOutputSpectro(loss=tensor(7.2967, grad_fn=<NllLossBackward0>), logits=tensor([[[ 0.5537,  0.4766,  0.2917,  ...,  0.3016, -0.5364, -0.4306],
         [ 0.9487, -0.1303, -0.1805,  ...,  0.1474, -0.1853, -0.8019],
         [ 0.5662,  0.3621, -0.3568,  ...,  0.0613, -0.3281, -0.0750],
         [ 0.7645,  0.4561, -0.1900,  ...,  0.2491, -0.7282, -1.3658],
         [ 0.0197,  1.1113, -0.7314,  ..., -0.4131, -0.7679, -1.1542],
         [ 0.7056,  0.2304, -0.5058,  ...,  0.4372, -0.3797, -0.7306]],

        [[-0.0434,  0.3159, -0.0291,  ...,  0.5430, -0.6446, -0.4710],
         [-0.2474,  0.8425, -0.3931,  ...,  0.2944, -0.0368,  0.2165],
         [-0.4595,  0.6255,  0.6558,  ...,  0.5852, -0.2850, -0.1359],
         [ 0.0050,  0.8261,  0.0548,  ...,  1.2372, -0.1472, -0.0383],
         [ 0.4785,  0.6806,  0.2442,  ...,  0.5867, -0.8200, -0.0069],
         [ 0.3526,  0.3159,  0.0327,  ...,  0.1641, -0.9132,  0.2976]]],
       grad_fn=<AddBackward0>), last_hidden_state=None, past_key_

In [130]:
model

BartSpektroForConditionalGeneration(
  (model): BartSpektroModel(
    (encoder): BartSpektroEncoder(
      (embed_tokens): Embedding(501, 1024, padding_idx=2)
      (embed_positions): BartSpektroLearnedPositionalEmbedding(12, 1024)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_